<a href="https://colab.research.google.com/github/100jy/voice_competition/blob/master/%EB%B3%B4%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#inception_with_l2_regulizer
def inception_l2_model(input_shape):
  def inception_module(x, o_1=64,r_3 =64, o_3 =128,r_5=16,o_5=32,pool=32):
    #size_1 filter
    x_1 = Conv2D(o_1,1,padding='same', kernel_regularizer=regularizers.l2(0.0005), bias_regularizer=regularizers.l2(0.0005))(x)
    '''
    #size_1 + size_1 filter
    x_2 = Conv2D(r_3,1,padding='same')(x)
    x_2 = Conv2D(o_3,1,padding='same')(x_2)
    '''
    #size_1 + size_5 filter
    x_3 = Conv2D(r_5,1,padding='same', kernel_regularizer=regularizers.l2(0.0005), bias_regularizer=regularizers.l2(0.0005))(x)
    x_3 = Conv2D(o_5,5,padding='same', kernel_regularizer=regularizers.l2(0.0005), bias_regularizer=regularizers.l2(0.0005))(x_3)

    #pooling
    x_4 = MaxPooling2D(pool_size = (3,3),strides =1,padding='same')(x)
    x_4 = Conv2D(pool, 1, padding='same')(x_4)

    return concatenate([x_1,x_3,x_4])

  #DeepCNN with Gelu
  def gelu(x):
      return 0.5*x*(1+tf.tanh(np.sqrt(2/np.pi)*(x+0.044715*tf.pow(x, 3))))


  inp = Input(input_shape) 

  x = Conv2D(64, (7, 7), strides = 2, padding = "same", kernel_regularizer=regularizers.l2(0.0005), bias_regularizer=regularizers.l2(0.0005))(inp)
  x = Activation(gelu)(x)
  x = MaxPooling2D((3, 3), padding = "same", strides = 2)(x)
  x = Conv2D(64, (1, 1), strides = 1, padding = "same", kernel_regularizer=regularizers.l2(0.0005), bias_regularizer=regularizers.l2(0.0005))(x)
  x = Activation(gelu)(x)
  x = Conv2D(192, (3, 3), strides = 1, padding = "same", kernel_regularizer=regularizers.l2(0.0005), bias_regularizer=regularizers.l2(0.0005))(x)
  x = Activation(gelu)(x)
  x = MaxPooling2D((3, 3), padding = "same", strides = 2)(x)

  x = inception_module(x, o_1=64, r_3=64, o_3=128, r_5=16, o_5=32, pool=32)
  x = inception_module(x, o_1=128, r_3=128, o_3=192, r_5=32, o_5=96, pool=64)
  x = MaxPooling2D(pool_size=(3, 3), strides=2, padding='same')(x)
  x = inception_module(x, o_1=192, r_3=96, o_3=208, r_5=16, o_5=48, pool=64)
  
  x = inception_module(x, o_1=160, r_3=112, o_3=224, r_5=24, o_5=64, pool=64)
  x = inception_module(x, o_1=128, r_3=128, o_3=256, r_5=24, o_5=64, pool=64)
  x = inception_module(x, o_1=112, r_3=144, o_3=288, r_5=32, o_5=64, pool=64)
  x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)

  x = MaxPooling2D(pool_size=(2, 2), strides=1, padding='same')(x)
  x = inception_module(x, o_1=256, r_3=160, o_3=320, r_5=32, o_5=128, pool=128)
  x = inception_module(x, o_1=384, r_3=192, o_3=384, r_5=48, o_5=128, pool=128)
  

  x = AveragePooling2D(pool_size=(2, 2), strides=3)(x)
  x = Conv2D(128, (1, 1),padding = "same", kernel_regularizer=regularizers.l2(0.0005), bias_regularizer=regularizers.l2(0.0005))(x)
  x = Activation(gelu)(x)
  x = Flatten()(x)
  #x = GlobalAveragePooling2D()(x)
  x = Dropout(0.8)(x)

  x = Dense(1000)(x)
  x = Activation(gelu)(x)
  x = Dropout(0.8)(x)
  output = Dense(30, activation = "softmax")(x)

  return Model(inp,output)

In [ ]:
#inception_resNet_V2
##model architecture
def Scaling_Residual(Inception, scale):
  x = Lambda(lambda Inception, scale : Inception * scale, arguments={'scale': scale})(Inception)
  x = Activation(activation = 'relu')(x)

  return x

def conv2d_bn(x,filters, kernel_size, padding = 'same', strides =(1,1), name = None, activation = 'relu'):
  x = Conv2D(filters, kernel_size, strides= strides,padding=padding, use_bias=False)(x)
  x = BatchNormalization(scale=False)(x)

  if activation != 'None':
   x = Activation(activation)(x)

  return x 

def stem(input_tensor, version = 'v2', name = None):
  if version == 'I-v4' or version == 'v2':
    x = conv2d_bn(input_tensor, 32,(3,3),padding='valid', strides = 2)
    x = conv2d_bn(x, 32,(3,3),padding='valid')
    x = conv2d_bn(x, 64,(3,3))

    branch_1 = MaxPooling2D((3,3),padding = 'valid', strides=2)(x)
    branch_2 = conv2d_bn(x,96,(3,3),padding = 'valid', strides=2)
    x = Concatenate()([branch_1,branch_2])

    branch_1 = conv2d_bn(x,64,(1,1))
    branch_1 = conv2d_bn(branch_1,96,(3,3),padding='valid')
    branch_2 = conv2d_bn(x,64,(1,1))
    branch_2 = conv2d_bn(branch_2,64,(7,1))
    branch_2 = conv2d_bn(branch_2,64,(1,7))
    branch_2 = conv2d_bn(branch_2,96,(3,3),padding='valid')
    x = Concatenate()([branch_1,branch_2])

    branch_1 = conv2d_bn(x,192,(3,3),padding = 'valid', strides=2)
    branch_2 = MaxPooling2D((3,3),padding = 'valid', strides=2)(x)
    x = Concatenate()([branch_1,branch_2])

    return x
 

reduction_table = {'v2' : [256,256,384,384]}



def Reduction_A(input_tensor, version='v2', name=None):
  k,l,m,n = reduction_table[version]

  branch_1 = MaxPooling2D((3,3),padding='valid',strides=2)(input_tensor)

  branch_2 = conv2d_bn(input_tensor, n, (3,3), padding  = 'valid', strides=2)

  branch_3 = conv2d_bn(input_tensor, k, (1,1))
  branch_3 = conv2d_bn(branch_3, l, (3,3))
  branch_3 = conv2d_bn(branch_3, m, (3,3), padding  = 'valid', strides=2)

  filter_concat = Concatenate()([branch_1,branch_2,branch_3])

  return filter_concat 
  
def Reduction_B(input_tensor, version='v2', name=None):

  branch_1 = MaxPooling2D((3,3),padding='valid',strides=2)(input_tensor)

  branch_2 = conv2d_bn(input_tensor, 256, (1,1))
  branch_2 = conv2d_bn(branch_2, 384, (3,3), padding  = 'valid', strides=2)

  branch_3 = conv2d_bn(input_tensor, 256, (1,1))
  branch_3 = conv2d_bn(branch_3, 288, (3,3), padding  = 'valid', strides=2)
  
  branch_4 = conv2d_bn(input_tensor, 256, (1,1))
  branch_4 = conv2d_bn(branch_4, 288, (3,3))
  branch_4 = conv2d_bn(branch_4, 320, (3,3), padding  = 'valid', strides=2)  

  filter_concat = Concatenate()([branch_1,branch_2,branch_3,branch_4])

  return filter_concat 



def Inception_ResNet_A(input_tensor, scale = 0.1, version='v2', name = None):
  if version == 'v1':
    raise ValueError
  
  else:
    branch_1 = conv2d_bn(input_tensor, 32, (1,1))

    branch_2 = conv2d_bn(input_tensor, 32, (1,1))
    branch_2 = conv2d_bn(branch_2, 32, (3,3))

    branch_3 = conv2d_bn(input_tensor, 32, (1,1))
    branch_3 = conv2d_bn(branch_3, 48, (3,3))
    branch_3 = conv2d_bn(branch_3, 64, (3,3))

    branches = Concatenate()([branch_1,branch_2,branch_3])
    Inception = conv2d_bn(branches, 384,(1,1), activation = 'None')

  scaled_activation = Scaling_Residual(Inception, scale = scale)
  residual_connection  = Add()([input_tensor, scaled_activation])

  return residual_connection

def Inception_ResNet_B(input_tensor, scale = 0.1, version='v2', name = None):
  if version == 'v1':
    raise ValueError
  
  else:
    branch_1 = conv2d_bn(input_tensor, 192, (1,1))

    branch_2 = conv2d_bn(input_tensor, 128, (1,1))
    branch_2 = conv2d_bn(branch_2, 160, (1,7))
    branch_2 = conv2d_bn(branch_2, 192, (7,1))

    branches = Concatenate()([branch_1,branch_2])
    Inception = conv2d_bn(branches, 1152,(1,1), activation = 'None')

  scaled_activation = Scaling_Residual(Inception, scale = scale)
  residual_connection  = Add()([input_tensor, scaled_activation])

  return residual_connection
  
def Inception_ResNet_C(input_tensor, scale = 0.1, version='v2', name = None):
  if version == 'v1':
    raise ValueError
  
  else:
    branch_1 = conv2d_bn(input_tensor, 192, (1,1))

    branch_2 = conv2d_bn(input_tensor, 192, (1,1))
    branch_2 = conv2d_bn(branch_2, 224, (1,3))
    branch_2 = conv2d_bn(branch_2, 256, (3,1))

    branches = Concatenate()([branch_1,branch_2])
    Inception = conv2d_bn(branches, 2144,(1,1), activation = 'None')

  scaled_activation = Scaling_Residual(Inception, scale = scale)
  residual_connection  = Add()([input_tensor, scaled_activation])

  return residual_connection


def Inception_ResNet(model_input, version = 'v2', classes = 30):
  x = stem(model_input, version = version)
  
  for i in range(5):
    x = Inception_ResNet_A(x, scale = 0.17, version =version)
 
  x = Reduction_A(x)
  
  for i in range(10):
    x = Inception_ResNet_B(x, scale = 0.1, version =version)  


  x = Reduction_B(x) 

  for i in range(5):
    x = Inception_ResNet_C(x, scale = 0.2, version =version)

  x = GlobalAveragePooling2D()(x)

  x = Dropout(0.8)(x)
  
  model_output = Dense(classes, activation = 'softmax')(x)
  model = Model(model_input,model_output)

  return model

In [ ]:
# Xception

def conv2d_bn(x, filters, kernel_size, padding='same', strides=1, activation='relu', weight_decay=1e-4):
    x = Conv2D(filters, kernel_size, padding=padding, strides=strides, kernel_regularizer=l2(weight_decay))(x)
    x = BatchNormalization()(x)
    
    if activation:
        x = Activation(activation)(x)
    
    return x

def sepconv2d_bn(x, filters, kernel_size, padding='same', strides=1, activation='relu', weight_decay=1e-4, depth_multiplier=1):
    x = SeparableConv2D(filters, kernel_size, padding=padding, strides=strides, depth_multiplier=depth_multiplier, depthwise_regularizer=l2(weight_decay), pointwise_regularizer=l2(weight_decay))(x)
    x = BatchNormalization()(x)
    
    if activation:
        x = Activation(activation)(x)
    
    return x

def Xception(model_input, classes):
    ## Entry flow
    x = conv2d_bn(model_input, 32, (3, 3), strides=2) # (299, 299, 3) -> (150, 150, 32)
    x = conv2d_bn(x, 64, (3, 3))

    for fliters in [128, 256, 728]: # (75, 75, 64) -> (75, 75, 128) -> (38, 38, 256) -> (19, 19, 728)
        residual = conv2d_bn(x, fliters, (1, 1), strides=2, activation=None)
        
        x = Activation(activation='relu')(x)
        x = sepconv2d_bn(x, fliters, (3, 3))
        x = sepconv2d_bn(x, fliters, (3, 3), activation=None)
        x = MaxPooling2D((3, 3), padding='same', strides=2)(x)
        
        x = Add()([x, residual])
        
        
    ## Middle flow
    for i in range(8): # (19, 19, 728)
        residual = x
        
        x = Activation(activation='relu')(x)
        x = sepconv2d_bn(x, 728, (3, 3))
        x = sepconv2d_bn(x, 728, (3, 3))
        x = sepconv2d_bn(x, 728, (3, 3), activation=None)
        
        x = Add()([x, residual])
        
        
    ## Exit flow
    residual = conv2d_bn(x, 1024, (1, 1), strides=2, activation=None) # (19, 19, 728) -> (10, 10, 1024)
        
    x = Activation(activation='relu')(x)
    x = sepconv2d_bn(x, 728, (3, 3))
    x = sepconv2d_bn(x, 1024, (3, 3), activation=None) # (19, 19, 728) -> (19, 19, 1024)
    x = MaxPooling2D((3, 3), padding='same', strides=2)(x) # (19, 19, 1024) -> (10, 10, 1024)
    
    x = Add()([x, residual])
    
    x = sepconv2d_bn(x, 1536, (3, 3))
    x = sepconv2d_bn(x, 2048, (3, 3))

    x = GlobalAveragePooling2D()(x)
    
    ## Optinal fully-connected layers
       
    '''
    x = Dense(4096)(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
 
    x = Dense(4096)(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)
    '''
    x = Dropout(0.8)(x)

    x = Dense(4096)(x)
    x = BatchNormalization()(x)
    x = Activation(activation='relu')(x)

    x = Dropout(0.8)(x)
    
    model_output = Dense(classes, activation='softmax')(x)

    model = Model(model_input, model_output, name='Xception')
    
    return model

In [ ]:
model_path = 'drive/My Drive/data/model_inception_high_leg_mel_spec/'
if not os.path.exists(model_path):
  os.mkdir(model_path)

# validattion 기준 모델 갱신
model_file = model_path + 'epoch_{epoch:03d}_val_{val_loss:3f}.hdf5'
checkpoint = ModelCheckpoint(filepath = model_file, monitor = 'val_loss', verbose = 1, save_best_only =True)

#lr 조정
lrschedule = LearningRateSchedule()

#10회간 validatation 좋아지지 않으면 early stop
early_stop = EarlyStopping(monitor = 'val_loss', patience = 5)

with strategy.scope():
  model = inception_l2_model(input_shape)
  model.compile(loss=tf.keras.losses.KLDivergence(), optimizer = 'adam')
  #model.summary()
  model.load_weights(glob('drive/My Drive/data/model_inception_high_leg_mel_spec/*.hdf5')[-1])
  history = model.fit(train_dataset, epochs = 100, validation_data=val_dataset, callbacks = [checkpoint, early_stop,lrschedule])

In [ ]:
#model compile
input_shape = (299,299,1)
model_input = Input(shape = input_shape)
optimizer = RMSprop(lr = 0.045, epsilon = 1.0, decay = 0.9)
#optimizer = Adam(0.045)
model_path = 'drive/My Drive/data/model_inceptionRes_mel_spec/'
if not os.path.exists(model_path):
  os.mkdir(model_path)

# validattion 기준 모델 갱신
model_file = model_path + 'epoch_{epoch:03d}_val_{val_loss:3f}.hdf5'
checkpoint = ModelCheckpoint(filepath = model_file, monitor = 'val_loss', verbose = 1, save_best_only =True)

#10회간 validatation 좋아지지 않으면 early stop
early_stop = EarlyStopping(monitor = 'val_loss', patience = 10)

#lr 조정
lrschedule = LearningRateSchedule()

with strategy.scope():
  model = Inception_ResNet(model_input)
  model.compile(optimizer, loss =tf.keras.losses.KLDivergence())
  #model.summary()
  #model.load_weights(glob('drive/My Drive/data/model_inceptionRes_mel_spec/*.hdf5')[-1])
  history = model.fit(train_dataset, epochs = 100, validation_data=val_dataset,callbacks = [checkpoint, early_stop])#,lrschedule

In [ ]:
#model compile
input_shape = (299,299,1)
model_input = Input(shape = input_shape)
optimizer = SGD(lr=0.01, momentum=0.9)

model_path = 'drive/My Drive/data/model_Xception_mel_spec/'
if not os.path.exists(model_path):
  os.mkdir(model_path)

# validattion 기준 모델 갱신
model_file = model_path + 'epoch_{epoch:03d}_val_{val_loss:3f}.hdf5'
checkpoint = ModelCheckpoint(filepath = model_file, monitor = 'val_loss', verbose = 1, save_best_only =True)

#10회간 validatation 좋아지지 않으면 early stop
early_stop = EarlyStopping(monitor = 'val_loss', patience = 10)

#lr 조정
lrschedule = LearningRateSchedule()

with strategy.scope():
  model_input = Input(shape=input_shape)
  model = Xception(model_input, 30)
  model.compile(optimizer, loss =tf.keras.losses.KLDivergence())
  #model.summary()
  #model.load_weights(glob('drive/My Drive/data/model_Xception_mel_spec/*.hdf5')[-1])
  history = model.fit(train_dataset, epochs = 100, validation_data=val_dataset,callbacks = [checkpoint, early_stop,lrschedule])